In [ ]:
!nvidia-smi -L

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

### Some basic setup, import:

In [ ]:
# %matplotlib inline

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import pandas as pd
import numpy as np
import os, json, cv2, random
import glob
# from matplotlib import pyplot as plt
from IPython import display
from IPython.display import Image
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model

import IPython
from PIL import Image
import itertools
from tabulate import tabulate

Set seed for reproducibility

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(seed=SEED)
torch.manual_seed(SEED)

## Datasets

In [ ]:
def print_dataset_instances(dataset_dicts, class_names):
    """
    Args:
        dataset_dicts (list[dict]): list of dataset dicts.
        class_names (list[str]): list of class names (zero-indexed).
    """
    num_classes = len(class_names)
    hist_bins = np.arange(num_classes + 1)
    histogram = np.zeros((num_classes,), dtype=np.int32)
    for entry in dataset_dicts:
        annos = entry["annotations"]
        classes = np.asarray(
            [x["category_id"] for x in annos if not x.get("iscrowd", 0)], dtype=np.int32
        )
        if len(classes):
            assert classes.min() >= 0, f"Got an invalid category_id={classes.min()}"
            assert (
                classes.max() < num_classes
            ), f"Got an invalid category_id={classes.max()} for a dataset of {num_classes} classes"
        histogram += np.histogram(classes, bins=hist_bins)[0]

    N_COLS = min(6, len(class_names) * 2)

    def short_name(x):
        # make long class names shorter. useful for lvis
        if len(x) > 13:
            return x[:11] + ".."
        return x

    data = list(
        itertools.chain(*[[short_name(class_names[i]), int(v)] for i, v in enumerate(histogram)])
    )
    total_num_instances = sum(data[1::2])
    data.extend([None] * (N_COLS - (len(data) % N_COLS)))
    if num_classes > 1:
        data.extend(["total", total_num_instances])
    data = itertools.zip_longest(*[data[i::N_COLS] for i in range(N_COLS)])
    table = tabulate(
        data,
        headers=["category", "#instances"] * (N_COLS // 2),
        tablefmt="pipe",
        numalign="left",
        stralign="center",
    )
    print("Distribution of instances among all {} categories:\n".format(num_classes))
    print(table)

In [ ]:
# print_dataset_instances(dataset_dicts, class_names)
# """
# Args:
#     dataset_dicts (list[dict]): list of dataset dicts.
#     class_names (list[str]): list of class names (zero-indexed).
# """

### Valid

In [ ]:
val_dataset_name = "BigScrewScrewWasherValid"
val_metadata = {}
val_json_file = "./data/screwwasherANDbigscrewvalid/result.json"
val_image_root = "./data/screwwasherANDbigscrewvalid"

register_coco_instances(val_dataset_name, val_metadata, val_json_file, val_image_root)
val_metadata = MetadataCatalog.get(val_dataset_name)
val_dataset_dicts = DatasetCatalog.get(val_dataset_name)

In [ ]:
val_dataset_name1 = "GlassMetalPaperPlasticValid"
val_metadata1 = {}
val_json_file1 = "./data/valid25/result.json"
val_image_root1 = "./data/valid25"

register_coco_instances(val_dataset_name1, val_metadata1, val_json_file1, val_image_root1)
val_metadata1 = MetadataCatalog.get(val_dataset_name1)
val_dataset_dicts1 = DatasetCatalog.get(val_dataset_name1)

In [ ]:
val_dataset_name2 = "ShowroomValid"
val_metadata2 = {}
val_json_file2 = "./data/showroomValid2/result.json"
val_image_root2 = "./data/showroomValid2"

register_coco_instances(val_dataset_name2, val_metadata2, val_json_file2, val_image_root2)
val_metadata2 = MetadataCatalog.get(val_dataset_name2)
val_dataset_dicts2 = DatasetCatalog.get(val_dataset_name2)

In [ ]:
val_dataset_name3 = "GlassMetalPaperPlasticAndShowroomValid"
val_metadata3 = {}
val_json_file3 = "./data/GlassMetalPaperPlasticAndShowroomValid/result.json"
val_image_root3 = "./data/GlassMetalPaperPlasticAndShowroomValid"

register_coco_instances(val_dataset_name3, val_metadata3, val_json_file3, val_image_root3)
val_metadata3 = MetadataCatalog.get(val_dataset_name3)
val_dataset_dicts3 = DatasetCatalog.get(val_dataset_name3)

In [ ]:
print_dataset_instances(dataset_dicts, metadata.thing_classes)

In [ ]:
print_dataset_instances(val_dataset_dicts1, val_metadata1.thing_classes)

In [ ]:
print_dataset_instances(val_dataset_dicts2, val_metadata2.thing_classes)

In [ ]:
print_dataset_instances(val_dataset_dicts3, val_metadata3.thing_classes)

### Train

In [ ]:
dataset_name = "BigScrewScrewWasher"
metadata = {}
json_file = "./data/screwwasherANDbigscrew/result.json"
image_root = "./data/screwwasherANDbigscrew"

register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
dataset_name1 = "glassMetalPapperPlastic"
metadata1 = {}
json_file1 = "./data/glassMetalPapperPlastic/result.json"
image_root1 = "./data/glassMetalPapperPlastic"

register_coco_instances(dataset_name1, metadata1, json_file1, image_root1)
metadata1 = MetadataCatalog.get(dataset_name1).set(thing_classes=["Glass", "Metal", "Paper", "Plastic"])
dataset_dicts1 = DatasetCatalog.get(dataset_name1)

In [ ]:
dataset_name2 = "showroom"
metadata2 = {}
json_file2 = "./data/showroom/result.json"
image_root2 = "./data/showroom"

register_coco_instances(dataset_name2, metadata2, json_file2, image_root2)
metadata2 = MetadataCatalog.get(dataset_name2)
dataset_dicts2 = DatasetCatalog.get(dataset_name2)

In [ ]:
dataset_name3 = "GlassMetalPaperPlasticAndShowroom"
metadata3 = {}
json_file3 = "./data/GlassMetalPaperPlasticAndShowroom/result.json"
image_root3 = "./data/GlassMetalPaperPlasticAndShowroom"

register_coco_instances(dataset_name3, metadata3, json_file3, image_root3)
metadata3 = MetadataCatalog.get(dataset_name3)
dataset_dicts3 = DatasetCatalog.get(dataset_name3)

In [ ]:
print_dataset_instances(val_dataset_dicts, val_metadata.thing_classes)

In [ ]:
print_dataset_instances(dataset_dicts1, metadata1.thing_classes)

In [ ]:
print_dataset_instances(dataset_dicts2, metadata2.thing_classes)

In [ ]:
print_dataset_instances(dataset_dicts3, metadata3.thing_classes)

In [ ]:
def imshow(img):
    _, ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [ ]:
def random_sample(dataset_dicts, number, metadata, save_image=False):
    i = 0
    for d in random.sample(dataset_dicts, number):
        i += 1
        img = cv2.imread(d["file_name"])
        visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
        vis = visualizer.draw_dataset_dict(d)
        imshow(vis.get_image()[:, :, ::-1])
        if save_image:        
            im = cv2.cvtColor(vis.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
            im = Image.fromarray(im)
            im.save("evaluation/" + str(i) + ".png")

In [ ]:
random_sample(val_dataset_dicts1, 5, val_metadata1, save_image=True)

In [ ]:
random_sample(val_dataset_dicts2, 5, val_metadata2, save_image=True)

In [ ]:
random_sample(val_dataset_dicts3, 5, val_metadata3, save_image=True)

In [ ]:
random_sample(dataset_dicts, 5, metadata, save_image=True)

# Inference & evaluation using the trained models
Let's run inference with the trained models on the validation datasets.



In [ ]:
def choose_model(output_path, model, epoch, dataset):
    weight_path = None
    dataset_yaml_file = None
    if model == "maskrcnn":
        directory_exists = os.path.exists(output_path + str(dataset) + "/" + str(epoch) + "/output_mask_rcnn_R_50_FPN_3x/")
        if directory_exists:
            weight_path = output_path + str(dataset) + "/" + str(epoch) + "/output_mask_rcnn_R_50_FPN_3x/"
            dataset_yaml_file = weight_path + "mask_rcnn_R_50_FPN_3x.yaml"
        else:
            print("Wrong model or epoch number or dataset!")
    elif model == "fasterrcnn":
        directory_exists = os.path.exists(output_path + str(dataset) + "/" + str(epoch) + "/output_faster_rcnn_R_50_FPN_3x/")
        if directory_exists:
            weight_path = output_path + str(dataset) + "/" + str(epoch) + "/output_faster_rcnn_R_50_FPN_3x/"
            dataset_yaml_file = weight_path + "faster_rcnn_R_50_FPN_3x.yaml"
        else:
            print("Wrong model or epoch number or dataset!")
    elif model == "retinanet":
        directory_exists = os.path.exists(output_path + str(dataset) + "/" + str(epoch) + "/output_retinanet_R_50_FPN_3x/")
        if directory_exists:
            weight_path = output_path + str(dataset) + "/" + str(epoch) + "/output_retinanet_R_50_FPN_3x/"
            dataset_yaml_file = weight_path + "retinanet_R_50_FPN_3x.yaml"
        else:
            print("Wrong model or epoch number or dataset!")  
            
    return weight_path, dataset_yaml_file

In [ ]:
def get_predictor(weight_path, dataset_yaml_file, threshold=0.7):
    cfg = get_cfg()
    cfg.merge_from_file(dataset_yaml_file)
    cfg.MODEL.WEIGHTS = os.path.join(weight_path, "model_final.pth")  # path to the model
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold   # set a custom testing threshold
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = threshold
    predictor = DefaultPredictor(cfg)

    return predictor

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=10000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

In [ ]:
def evaluate(image, metadata, predictor, instance_mode=False, save_image=False):
    im = cv2.imread(image)
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    
    v = {}
    if instance_mode:
        v = Visualizer(im[:, :, ::-1],
                        metadata=metadata, 
                        scale=0.5, 
                        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
        )
    else:
        v = Visualizer(im[:, :, ::-1],
                        metadata=metadata, 
                        scale=0.5,
        )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    if save_image:        
        im = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
        im = Image.fromarray(im)
        im.save("evaluation/" +  str(image.rsplit('\\', 1)[1]))
#         im.save("evaluation/" +  str(image.rsplit('/', 1)[1]))

    imshow(out.get_image()[:, :, ::-1])

In [ ]:
evaluate("./data/dataset-resized/glass/test/glass458.jpg", val_metadata1, predictor, instance_mode=True, save_image=True)

In [ ]:
evaluate("./data/dataset-resized/glass/test/glass458.jpg", val_metadata1, predictor, instance_mode=True)

In [ ]:
evaluate("./data/dataset-resized/glass/test/glass458.jpg", val_metadata1, predictor, instance_mode=False)

In [ ]:
def random_test_sample(dataset_dicts, number, metadata, predictor, instance_mode=False, save_image=False):
    for i in range(number):
        images = glob.glob(dataset_dicts + "*.png")
#         images = glob.glob(dataset_dicts + "*.jpg")
        random_image = random.choice(images)
        evaluate(random_image, metadata, predictor, instance_mode, save_image)

In [ ]:
test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=10000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/valid25/images/"
# test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor, save_image=True)

In [ ]:
output_path = "./detectron2/output/"
weight_path = 
dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomValid2/images/"
random_test_sample(test_dataset_path, 5, val_metadata2, predictor, save_image=True)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=10000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/GlassMetalPaperPlasticAndShowroomValid/images/"
random_test_sample(test_dataset_path, 5, val_metadata3, predictor, save_image=True)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/test/"
random_test_sample(test_dataset_path, 10, val_metadata2, predictor, save_image=True)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomValid2/images/"
# test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata2, predictor, save_image=True)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomValid2/images/"
# test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata2, predictor, save_image=True)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/test/"
random_test_sample(test_dataset_path, 10, val_metadata2, predictor, save_image=True)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/test/"
random_test_sample(test_dataset_path, 10, val_metadata2, predictor, save_image=True)

In [ ]:
dataset_yaml_file = "./detectron2/output/BulkGoods/10000/BigScrewANDScrewWasher/output_mask_rcnn_R_50_FPN_3x/model_mask_rcnn_R_50_FPN_3x.yaml"
weight_path = "./detectron2/output/BulkGoods/10000/BigScrewANDScrewWasher/output_mask_rcnn_R_50_FPN_3x/"
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/screwwasherANDbigscrewvalid/"
random_test_sample(test_dataset_path, 10, metadata, predictor, save_image=True)

In [ ]:
def inference(dataset_name, dataset_yaml_file, weight_path, model_pth="model_final.pth", threshold=0.7):
    cfg = get_cfg()
    cfg.merge_from_file(dataset_yaml_file)
    cfg.MODEL.WEIGHTS = os.path.join(weight_path, model_pth)  # path to the model
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold   # set a custom testing threshold
    predictor = DefaultPredictor(cfg)
    evaluator = COCOEvaluator(dataset_name, output_dir="./output")
    val_loader = build_detection_test_loader(cfg, dataset_name)
    evaluation_results = inference_on_dataset(predictor.model, val_loader, evaluator)
    return evaluation_results

In [ ]:
dataset_yaml_file = "./detectron2/output/Showroom/30000/output_mask_rcnn_R_50_FPN_3x/mask_rcnn_R_50_FPN_3x.yaml"
weight_path = "./detectron2/output/Showroom/30000/output_mask_rcnn_R_50_FPN_3x/"
model_pth = "model_final.pth"
threshold = 0.7
dataset_name = val_dataset_name2

evaluation_results = inference(dataset_name, dataset_yaml_file, weight_path, model_pth, threshold)

In [ ]:
evaluation_results

In [ ]:
output_path = "./detectron2/output/"
models = ["maskrcnn", "fasterrcnn", "retinanet"]
epoch = [10000, 30000, 50000, 80000]
datasets = ["Downloaded", "Showroom", "DownloadedAndShowroom"]

result = []
for i in models:
    for j in epoch:
        for k in datasets:
            segmentation = None
            if i == "maskrcnn":
                directory_exists = os.path.exists(output_path + str(k) + "/" + str(j) + "/output_mask_rcnn_R_50_FPN_3x/")
                if directory_exists:
                    weight_path = output_path + str(k) + "/" + str(j) + "/output_mask_rcnn_R_50_FPN_3x/"
                    dataset_yaml_file = weight_path + "mask_rcnn_R_50_FPN_3x.yaml"
                    segmentation = True
            elif i == "fasterrcnn":
                directory_exists = os.path.exists(output_path + str(k) + "/" + str(j) + "/output_faster_rcnn_R_50_FPN_3x/")
                if directory_exists:
                    weight_path = output_path + str(k) + "/" + str(j) + "/output_faster_rcnn_R_50_FPN_3x/"
                    dataset_yaml_file = weight_path + "faster_rcnn_R_50_FPN_3x.yaml"
                    segmentation = False
            elif i == "retinanet":
                directory_exists = os.path.exists(output_path + str(k) + "/" + str(j) + "/output_retinanet_R_50_FPN_3x/")
                if directory_exists:
                    weight_path = output_path + str(k) + "/" + str(j) + "/output_retinanet_R_50_FPN_3x/"
                    dataset_yaml_file = weight_path + "retinanet_R_50_FPN_3x.yaml"
                    segmentation = False
            
            model_pth = "model_final.pth"
            threshold = 0.7
            
            if k == "Downloaded":
                dataset_name = val_dataset_name1
            elif k == "Showroom":
                dataset_name = val_dataset_name2
            elif k == "DownloadedAndShowroom":
                dataset_name = val_dataset_name3

            if directory_exists:
                evaluation_results = inference(dataset_name, dataset_yaml_file, weight_path, model_pth, threshold)            
                result.append([i, j, k, segmentation, evaluation_results])                   

In [ ]:
def save_result(result):
    df = pd.DataFrame()
    for i in range(len(result)):
        temp = pd.DataFrame(result[i][4], columns=result[i][4].keys())
        temp = temp.T
        
        temp = temp.drop(['APs', 'APm', 'APl', 'AR1', 'AR10'], axis=1)

        segmentation = result[i][3]

        if segmentation:
            model = pd.Series([result[i][0], result[i][0]], index=[0, 1])
            epoch = pd.Series([result[i][1], result[i][1]], index=[0, 1])
            dataset = pd.Series([result[i][2], result[i][2]], index=[0, 1])
        else:
            model = pd.Series([result[i][0]], index=[0])
            epoch = pd.Series([result[i][1]], index=[0])
            dataset = pd.Series([result[i][2]], index=[0])

        temp['model'] = model.values
        temp['epoch'] = epoch.values
        temp['dataset'] = dataset.values

        df = pd.concat([df, temp])
        df.to_csv('result/train_evaulation.csv', index=True, index_label="Result-type") 
    
    return df

In [ ]:
print_dataset_instances(val_dataset_dicts1, val_metadata1.thing_classes)

In [ ]:
print_dataset_instances(val_dataset_dicts2, val_metadata2.thing_classes)

In [ ]:
print_dataset_instances(val_dataset_dicts3, val_metadata3.thing_classes)

In [ ]:
df = save_result(result)

In [ ]:
df

### Check result in resolution

In [ ]:
# Import class time from time module
from time import time

In [ ]:
def resize_with_aspect_ratio(image, width=None, height=None):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return dim

In [ ]:
def evaluate2(image, metadata, predictor, width=1280, instance_mode=False, save_image=False):
    start = int(time() * 1000)
    
    im = cv2.imread(image)
    print("Original image shape: " + str(im.shape))
    
    downsize = resize_with_aspect_ratio(im, width=width)
    img = cv2.resize(im, dsize=downsize, interpolation=cv2.INTER_AREA)
    print("Resized image shape: " + str(img.shape))
    
    startPredictor = int(time() * 1000)
    outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    endPredictor = int(time() * 1000)
    
#     print(outputs)
    
    v = {}
    if instance_mode:
        v = Visualizer(img[:, :, ::-1],
                        metadata=metadata, 
                        scale=0.5, 
                        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
        )
    else:
        v = Visualizer(img[:, :, ::-1],
                        metadata=metadata, 
                        scale=0.5,
        )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    if save_image:        
        im = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
        im = Image.fromarray(im)
        im.save(image.rsplit('/', 1)[1])

    imshow(out.get_image()[:, :, ::-1])
    
    end = int(time() * 1000)
    
    print("The whole process " + str(end-start) + " ms")
    print("The prediction time " + str(endPredictor-startPredictor) + " ms")

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

In [ ]:
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)
evaluate2("./data/test/200.png", val_metadata2, predictor, width=2448, instance_mode=False, save_image=False)

In [ ]:
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)
evaluate2("./data/test/200.png", val_metadata2, predictor, width=2000, instance_mode=False, save_image=False)

In [ ]:
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)
evaluate2("./data/test/200.png", val_metadata2, predictor, width=1600, instance_mode=False, save_image=False)

In [ ]:
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)
evaluate2("./data/test/200.png", val_metadata2, predictor, width=1280, instance_mode=False, save_image=False)

In [ ]:
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)
evaluate2("./data/test/200.png", val_metadata2, predictor, width=1000, instance_mode=False, save_image=False)

In [ ]:
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)
evaluate2("./data/test/200.png", val_metadata2, predictor, width=720, instance_mode=False, save_image=False)

In [ ]:
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)
evaluate2("./data/test/200.png", val_metadata2, predictor, width=500, instance_mode=False, save_image=False)

In [ ]:
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)
evaluate2("./data/test/200.png", val_metadata2, predictor, width=400, instance_mode=False, save_image=False)

In [ ]:
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)
evaluate2("./data/test/200.png", val_metadata2, predictor, width=200, instance_mode=False, save_image=False)

### COCO-pretrained R50-FPN Mask R-CNN

#### Learned on downloaded dataset

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=10000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=30000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=80000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

#### Learned on showroom dataset

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomImagesResized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

#### Learned on showroom and downloaded dataset

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=10000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=10000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomImagesResized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=30000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="maskrcnn", epoch=30000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomImagesResized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

### COCO-pretrained R50-FPN Faster R-CNN

#### Learned on downloaded dataset

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=10000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=30000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=80000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

#### Learned on showroom dataset

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomImagesResized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

#### Learned on showroom and downloaded dataset

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=10000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=10000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomImagesResized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=30000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="fasterrcnn", epoch=30000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomImagesResized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

### COCO-pretrained R50-FPN RetinaNet

#### Learned on downloaded dataset

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=10000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=30000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=80000, dataset="Downloaded")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

#### Learned on showroom dataset

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=30000, dataset="Showroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomImagesResized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

#### Learned on showroom and downloaded dataset

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=10000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=10000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomImagesResized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=30000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)

In [ ]:
output_path = "./detectron2/output/"
weight_path, dataset_yaml_file = choose_model(output_path, model="retinanet", epoch=30000, dataset="DownloadedAndShowroom")
predictor = get_predictor(weight_path, dataset_yaml_file, threshold=0.7)

test_dataset_path = "./data/showroomImagesResized/test/"
random_test_sample(test_dataset_path, 5, val_metadata1, predictor)